In [1]:
import numpy as np
import scipy.io as scio
import argparse
import tqdm
import pandas as pd
from typing import List, Any, Tuple
from scipy.stats import multivariate_normal, rv_continuous
from numpy import ndarray

In [2]:
# Hàm đọc file matlab
def get_GNBG_from_path(path):
    data = scio.loadmat(path)
    GNBG = data['GNBG']
    """
    Các thuộc tính của GNBG:
    - OptimumValue: Giá trị tối ưu
    - o: Số thanh phần
    - Dimension: Số chiều
    - MinCoordinate: Giới hạn dưới
    - MaxCoordinate: Giới hạn trên
    """
    return data['GNBG']

# Hàm ma trận xoay
def Transform(X, Alpha, Beta):
    Y = X.copy()
    tmp = (X > 0)
    Y[tmp] = np.log(X[tmp])
    Y[tmp] = np.exp(Y[tmp] + Alpha[0] * (np.sin(Beta[0] * Y[tmp]) + np.sin(Beta[1] * Y[tmp])))

    tmp = (X < 0)
    Y[tmp] = np.log(-X[tmp])
    Y[tmp] = -np.exp(Y[tmp] + Alpha[1] * (np.sin(Beta[2] * Y[tmp]) + np.sin(Beta[3] * Y[tmp])))

    return Y

# Hàm tính toán giá trị thích nghi của cá thể
def fitness_of_ind(X: ndarray, GNBG: Any) -> float:

    # Đọc file GNBG và lấy ra tham số
    num_com = GNBG['o'][0][0][0][0]
    dim = GNBG['Dimension'][0][0][0][0]
    min_pos = GNBG['Component_MinimumPosition'][0][0].reshape(num_com, dim)
    rot_mat = GNBG['RotationMatrix'][0][0].reshape(dim,dim,num_com)
    sigma = GNBG['ComponentSigma'][0,0].reshape(num_com,1)
    hehe = GNBG['Component_H'][0,0].reshape(num_com,dim)
    mu = GNBG['Mu'][0][0].reshape(num_com,2)
    omega = GNBG['Omega'][0][0].reshape(num_com,4)
    lamb = GNBG['lambda'][0][0].reshape(num_com,1)

    # Tính toán fitness
    x = X.reshape(-1,1)
    f = np.empty((1, num_com))
    for k in range(num_com):
        inp = x - min_pos[k, :].reshape(-1,1)
        a = Transform( inp.T  @  rot_mat[:,:,k],mu[k, :],omega[k,:])
        b = Transform(rot_mat[:,:,k] @ inp, mu[k, :], omega[k, :] )
        f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]
    result = np.min(f)
    return result

In [3]:
# Hàm khớp phân phối
def fit_multivariate_gaussian(population: ndarray) -> rv_continuous:
    """
    Khớp phân phối Gaussian đa biến cho quần thể.
    :param population: Quần thể muốn khớp phân phối.
    :return: Mô hình Gaussian đa biến.
    """
    noisy_pop = population + np.random.normal(0, 0.1, population.shape)
    mean = np.mean(noisy_pop, axis=0)
    cov = np.cov(noisy_pop.T)
    return multivariate_normal(mean=mean, cov=cov)

In [4]:
data_paths = [f'/kaggle/input/gnbg-functions/f{i}.mat' for i in range(1, 25)]

def get_data_from_path(path):
    data = scio.loadmat(path)
    GNBG = data['GNBG']
    return {
        "Optimal value": GNBG['OptimumValue'][0][0][0][0],
        "Num components": GNBG['o'][0][0][0][0],
        "Dimension": GNBG['Dimension'][0][0][0][0],
        "LB": GNBG['MinCoordinate'][0][0][0][0],
        "UB": GNBG['MaxCoordinate'][0][0][0][0]
    }

In [5]:
tasks_1 = [get_GNBG_from_path(data_paths[i]) for i in range(6)]

df_1 = pd.DataFrame([get_data_from_path(data_paths[i]) for i in range(6)], index=[f'f{i + 1}' for i in range(6)])

df_1

,Optimal value,Num components,Dimension,LB,UB
f1,-1081.983799,1,30,-100,100
f2,-703.132815,1,30,-100,100
f3,-357.579750,1,30,-100,100
f4,-382.620521,1,30,-100,100
f5,-337.508998,1,30,-100,100
f6,-186.864053,1,30,-100,100


In [6]:
tasks_2 = [get_GNBG_from_path(data_paths[i]) for i in range(6, 15)]

df_2 = pd.DataFrame([get_data_from_path(data_paths[i]) for i in range(6, 15)], index=[f'f{i + 1}' for i in range(6, 15)])

df_2

,Optimal value,Num components,Dimension,LB,UB
f7,-912.857374,1,30,-100,100
f8,-656.788998,1,30,-100,100
f9,-884.736010,1,30,-100,100
f10,-604.974827,1,30,-100,100
f11,-118.075358,1,30,-100,100
f12,-1002.479079,1,30,-100,100
f13,-216.727696,1,30,-100,100
f14,-194.038194,1,30,-100,100
f15,-234.280428,1,30,-100,100


In [7]:
tasks_3 = [get_GNBG_from_path(data_paths[i]) for i in range(15, 24)]

df_3 = pd.DataFrame([get_data_from_path(data_paths[i]) for i in range(15, 24)], index=[f'f{i + 1}' for i in range(15, 24)])

df_3

,Optimal value,Num components,Dimension,LB,UB
f16,-5000.000000,5,30,-100,100
f17,-5000.000000,5,30,-100,100
f18,-5000.000000,5,30,-100,100
f19,-5000.000000,5,30,-100,100
f20,-98.928026,5,30,-100,100
f21,-50.000000,5,30,-100,100
f22,-1000.000000,2,30,-100,100
f23,-100.000000,5,30,-100,100
f24,-98.901653,5,30,-100,100


In [8]:
class MTEAD:
    def __init__(self, num_tasks: int, pop_size: int, dimensions: int, tasks: List[Any], bounds: Tuple[float, float], F: float, CR: float, generations: int, transfer_prob: float):
        self.num_tasks = num_tasks
        self.pop_size = pop_size
        self.dimensions = dimensions
        self.tasks = tasks
        self.bounds = bounds
        self.F = F
        self.CR = CR
        self.generations = generations
        self.transfer_prob = transfer_prob
        self.populations = self.initialize_population()
        self.epsilons = [1.0 for _ in range(num_tasks)]
        self.best_fitnesses = [np.inf for _ in range(num_tasks)]

    def initialize_population(self) -> List[ndarray]:
        LB, UB = self.bounds
        return [np.random.uniform(LB, UB, size=(self.pop_size, self.dimensions)) for _ in range(self.num_tasks)]

    def create_offsprings(self, population: ndarray) -> ndarray:
        num_individuals, dimensions = population.shape
        offspring = np.empty_like(population)
        LB, UB = self.bounds

        for i in range(num_individuals):
            idxs = [idx for idx in range(num_individuals) if idx != i]
            r1, r2, r3 = population[np.random.choice(idxs, 3, replace=False)]
            mutant = r1 + self.F * (r2 - r3)

            crossover_mask = np.random.rand(dimensions) < self.CR
            trial = np.where(crossover_mask, mutant, population[i])

            offspring[i] = np.clip(trial, LB, UB)

        return offspring

    def select_transfer_candidates(self, task_index: int) -> ndarray:
        current_population = self.populations[task_index]
        gaussian_model = fit_multivariate_gaussian(current_population)

        candidates = np.vstack([self.populations[j] for j in range(self.num_tasks) if j != task_index])
        probabilities = gaussian_model.pdf(candidates)

        epsilon_i = self.epsilons[task_index]
        num_candidates = int(epsilon_i * self.pop_size * (self.num_tasks - 1))
        candidate_indices = np.argsort(probabilities)[:num_candidates]

        return candidates[candidate_indices]

    def adapt_parameters(self, task_index: int) -> Tuple[ndarray, float]:
        population = self.populations[task_index]
        offsprings = self.create_offsprings(population)
        transfer_candidates = self.select_transfer_candidates(task_index)

        combined_population = np.vstack([population, offsprings, transfer_candidates])
        combined_fitness = np.array([fitness_of_ind(ind, self.tasks[task_index]) for ind in combined_population])

        sorted_indices = np.argsort(combined_fitness)
        population_updated = combined_population[sorted_indices[:self.pop_size]]

        start_index_transfer_candidates = len(population) + len(offsprings)
        successful_transfers = sum(idx < self.pop_size for idx in sorted_indices if idx >= start_index_transfer_candidates)
        epsilon_updated = successful_transfers / len(transfer_candidates) if len(transfer_candidates) > 0 else self.epsilons[task_index]

        return population_updated, epsilon_updated

    def run(self):
        for _ in tqdm.tqdm(range(self.generations)):
            for i in range(self.num_tasks):
                if np.random.rand() < self.transfer_prob:
                    self.populations[i], self.epsilons[i] = self.adapt_parameters(i)
                else:
                    offspring = self.create_offsprings(self.populations[i])
                    combined_population = np.vstack([self.populations[i], offspring])
                    combined_fitness = np.array([fitness_of_ind(ind, self.tasks[i]) for ind in combined_population])
                    sorted_indices = np.argsort(combined_fitness)[:self.pop_size]
                    self.populations[i] = combined_population[sorted_indices]

                best_current_fitness = fitness_of_ind(self.populations[i][0], self.tasks[i])
                if best_current_fitness < self.best_fitnesses[i]:
                    self.best_fitnesses[i] = best_current_fitness

        return self.populations, self.epsilons, self.best_fitnesses

In [9]:
mtead_1 = MTEAD(
    num_tasks = 6,
    pop_size = 100,
    dimensions = 30,
    tasks = tasks_1,
    bounds = (-100, 100),
    F = 0.5,
    CR = 0.8,
    generations = 3000,
    transfer_prob = 0.2
)

_, _, best_fitnesses_1 = mtead_1.run()

df_1['Optimization Results'] = best_fitnesses_1

df_1

  0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_18/1195317327.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]
100%|██████████| 3000/3000 [11:17<00:00,  4.43it/s]


,Optimal value,Num components,Dimension,LB,UB,Optimization Results
f1,-1081.983799,1,30,-100,100,-1081.983799
f2,-703.132815,1,30,-100,100,-703.132815
f3,-357.579750,1,30,-100,100,-357.579750
f4,-382.620521,1,30,-100,100,-382.495333
f5,-337.508998,1,30,-100,100,-335.918359
f6,-186.864053,1,30,-100,100,-185.066421


In [10]:
mtead_2 = MTEAD(
    num_tasks = 9,
    pop_size = 100,
    dimensions = 30,
    tasks = tasks_2,
    bounds = (-100, 100),
    F = 0.9,
    CR = 0.6,
    generations = 3000,
    transfer_prob = 0.2
)

_, _, best_fitnesses_2 = mtead_2.run()

df_2['Optimization Results'] = best_fitnesses_2

df_2

  0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_18/1195317327.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]
100%|██████████| 3000/3000 [17:06<00:00,  2.92it/s]


,Optimal value,Num components,Dimension,LB,UB,Optimization Results
f7,-912.857374,1,30,-100,100,-907.598449
f8,-656.788998,1,30,-100,100,-652.061252
f9,-884.736010,1,30,-100,100,4006.634234
f10,-604.974827,1,30,-100,100,-532.485371
f11,-118.075358,1,30,-100,100,-117.593953
f12,-1002.479079,1,30,-100,100,-1002.260682
f13,-216.727696,1,30,-100,100,-215.974084
f14,-194.038194,1,30,-100,100,-165.131522
f15,-234.280428,1,30,-100,100,-231.167771


In [11]:
mtead_3 = MTEAD(
    num_tasks = 9,
    pop_size = 100,
    dimensions = 30,
    tasks = tasks_3,
    bounds = (-100, 100),
    F = 0.9,
    CR = 0.5,
    generations = 3000,
    transfer_prob = 0.2
)

_, _, best_fitnesses_3 = mtead_3.run()

df_3['Optimization Results'] = best_fitnesses_3

df_3

  0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_18/1195317327.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]
100%|██████████| 3000/3000 [44:57<00:00,  1.11it/s]


,Optimal value,Num components,Dimension,LB,UB,Optimization Results
f16,-5000.000000,5,30,-100,100,-4317.900000
f17,-5000.000000,5,30,-100,100,-4989.659627
f18,-5000.000000,5,30,-100,100,-4999.971917
f19,-5000.000000,5,30,-100,100,-4999.986867
f20,-98.928026,5,30,-100,100,-99.553048
f21,-50.000000,5,30,-100,100,-49.577928
f22,-1000.000000,2,30,-100,100,-999.759211
f23,-100.000000,5,30,-100,100,-99.798800
f24,-98.901653,5,30,-100,100,-93.465552
